In [2]:
%pylab inline
import numpy as np
import pylab as pl


Welcome to pylab, a matplotlib-based Python environment [backend: module://IPython.kernel.zmq.pylab.backend_inline].
For more information, type 'help(pylab)'.


In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans

In [5]:
twenty_train = fetch_20newsgroups(subset='train')
vec = TfidfVectorizer(max_df=0.5)
X_train = vec.fit_transform(twenty_train.data)
y_train = twenty_train.target

In [12]:
kmeans = MiniBatchKMeans(n_clusters=1000, batch_size=1000, reassignment_ratio=0,
                         n_init=1, max_iter=10, compute_labels=False, verbose=10)
%time kmeans.fit(X_train)

Init 1/1 with method: k-means++
Inertia for init 1/1: 1692.760402
Minibatch iteration 1/120:mean batch inertia: 0.860088, ewa inertia: 0.860088 
Minibatch iteration 2/120:mean batch inertia: 0.814616, ewa inertia: 0.852051 
Minibatch iteration 3/120:mean batch inertia: 0.818794, ewa inertia: 0.846173 
Minibatch iteration 4/120:mean batch inertia: 0.802534, ewa inertia: 0.838459 
Minibatch iteration 5/120:mean batch inertia: 0.791585, ewa inertia: 0.830174 
Minibatch iteration 6/120:mean batch inertia: 0.805043, ewa inertia: 0.825732 
Minibatch iteration 7/120:mean batch inertia: 0.806815, ewa inertia: 0.822388 
Minibatch iteration 8/120:mean batch inertia: 0.778904, ewa inertia: 0.814702 
Minibatch iteration 9/120:mean batch inertia: 0.794126, ewa inertia: 0.811065 
Minibatch iteration 10/120:mean batch inertia: 0.796617, ewa inertia: 0.808511 
Minibatch iteration 11/120:mean batch inertia: 0.796310, ewa inertia: 0.806355 
Minibatch iteration 12/120:mean batch inertia: 0.786515, ewa in

MiniBatchKMeans(batch_size=1000, compute_labels=False, init='k-means++',
        init_size=None, max_iter=10, max_no_improvement=10,
        n_clusters=1000, n_init=1, random_state=None, reassignment_ratio=0,
        tol=0.0, verbose=10)

In [13]:
X_proj_train = np.dot(X_train, kmeans.cluster_centers_.T)

ValueError: setting an array element with a sequence.

In [14]:
from sklearn.metrics.pairwise import safe_sparse_dot

In [16]:
X_train_proj = safe_sparse_dot(X_train, kmeans.cluster_centers_.T)

In [21]:
X_train_proj.shape

(11314, 1000)

In [18]:
from sklearn.ensemble import ExtraTreesClassifier

In [32]:
%time clf = ExtraTreesClassifier(n_estimators=100, max_depth=100, n_jobs=-1, oob_score=True, bootstrap=True).fit(X_train_proj, y_train)

CPU times: user 13.3 s, sys: 2.13 s, total: 15.5 s
Wall time: 2min 5s


In [33]:
clf.oob_score_

0.67668375464026864

In [28]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.grid_search import GridSearchCV

In [29]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
gs = GridSearchCV(PassiveAggressiveClassifier(), param_grid, cv=3, n_jobs=-1)
%time gs.fit(X_train_proj, y_train)

CPU times: user 4.03 s, sys: 2.87 s, total: 6.9 s
Wall time: 17.7 s


GridSearchCV(cv=3,
       estimator=PassiveAggressiveClassifier(C=1.0, fit_intercept=True, loss='hinge', n_iter=5,
              n_jobs=1, random_state=None, shuffle=False, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid={'C': [0.01, 0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, score_func=None, scoring=None, verbose=0)

In [30]:
gs.best_params_

{'C': 1}

In [31]:
gs.best_score_

0.83206646632490722